In [1]:
%config IPCompleter.greedy=True

In [2]:
import sys
import torch

In [3]:
sys.path.append("../")
sys.argv = ["foo","-r","mini"]

In [4]:
import modules

In [5]:
import Driver
from Driver import logging
msg=logging.getLogger("Train")

In [6]:
import NN_driver
NN_driver.load_NN_variables()

In [7]:
num_workers = NN_driver.num_workers
batch_size = NN_driver.batch_size
split_fraction = NN_driver.split_fraction
epochs = NN_driver.epochs
lr = NN_driver.lr
weight_decay=NN_driver.weight_decay
class_names=NN_driver.class_names
num_classes = len(NN_driver.class_names)
doReWeight=NN_driver.doReWeight
doWeighted=NN_driver.doWeighted

In [8]:
%load_ext tensorboard
%tensorboard --logdir lightening_logs/

Reusing TensorBoard on port 6006 (pid 58672), started 0:02:34 ago. (Use '!kill 58672' to kill it.)

In [10]:
from pytorch_lightning import Trainer
import modules
model = modules.CatL(num_classes = num_classes, learning_rate=lr, batch_size=batch_size, weight_decay=weight_decay, split_fraction=split_fraction)
trainer = Trainer(gpus=1, max_epochs = 2, num_sanity_val_steps=0)
trainer.fit(model)

DatasetLoader :: INFO     ::       Attempt to load:/nfs/dust/atlas/user/hteagle/PyAnalysisUtils/ML/NN/data/processed/data_Wh_21_2_112_reco_sig.pt


[]


GPU available: True, used: True
lightning    :: INFO     ::       GPU available: True, used: True
TPU available: False, using: 0 TPU cores
lightning    :: INFO     ::       TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]
lightning    :: INFO     ::       CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type         | Params
-----------------------------------------------
0 | dropout10     | AlphaDropout | 0     
1 | dropout20     | AlphaDropout | 0     
2 | dropout30     | AlphaDropout | 0     
3 | linear_input  | Linear       | 5 K   
4 | linear1       | Linear       | 19 K  
5 | linear2       | Linear       | 19 K  
6 | linear3       | Linear       | 19 K  
7 | linear4       | Linear       | 19 K  
8 | linear5       | Linear       | 19 K  
9 | linear_output | Linear       | 705   
lightning    :: INFO     ::       
  | Name          | Type         | Params
-----------------------------------------------
0 | dropout10     | AlphaDropout | 0     
1 | dropout20     | Al

Epoch 1:  80%|███████▉  | 43/54 [01:44<00:26,  2.42s/it, loss=1.496, v_num=15]
Validating: 0it [00:00, ?it/s]
Epoch 1:  81%|████████▏ | 44/54 [01:46<00:24,  2.43s/it, loss=1.496, v_num=15]

../modules.py:109: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = torch.nn.functional.softmax(x)## When training/calculating losses using CrossEntropyLoss we should feed it raw logits, it applies softmax itself. When applying/getting probabilites, we should switch this on..annoying



Epoch 2:  80%|███████▉  | 43/54 [01:43<00:26,  2.41s/it, loss=0.716, v_num=15]
Validating: 0it [00:00, ?it/s]
Epoch 2: 100%|██████████| 54/54 [02:09<00:00,  2.39s/it, loss=0.716, v_num=15]


1

In [11]:
logger_path = trainer.logger.save_dir+"/"+trainer.logger.name+"/version_"+str(trainer.logger.version)+"/"
logger_path

'/nfs/dust/atlas/user/hteagle/PyAnalysisUtils/ML/NN/notebooks/lightning_logs/version_15/'

Get the model back in its notmal format

In [12]:
torch.save(model.state_dict(), logger_path+"model.pt")
from dataset import bbMeT_NN
dataset = bbMeT_NN(root='data', device="cpu", transform=None)
ptmodel = modules.Cat(X_size=len(dataset.X_data[0]),num_classes=num_classes)
ptmodel.load_state_dict(torch.load(logger_path+"model.pt"))
ptmodel.eval()

DatasetLoader :: INFO     ::       Attempt to load:/nfs/dust/atlas/user/hteagle/PyAnalysisUtils/ML/NN/data/processed/data_Wh_21_2_112_reco_sig.pt


[]


Cat(
  (dropout10): AlphaDropout(p=0.1, inplace=False)
  (dropout20): AlphaDropout(p=0.2, inplace=False)
  (dropout30): AlphaDropout(p=0.3, inplace=False)
  (linear_input): Linear(in_features=35, out_features=140, bias=True)
  (linear1): Linear(in_features=140, out_features=140, bias=True)
  (linear2): Linear(in_features=140, out_features=140, bias=True)
  (linear3): Linear(in_features=140, out_features=140, bias=True)
  (linear4): Linear(in_features=140, out_features=140, bias=True)
  (linear5): Linear(in_features=140, out_features=140, bias=True)
  (linear_output): Linear(in_features=140, out_features=5, bias=True)
)

In [13]:
import XGB_board
import importlib
importlib.reload(XGB_board)
board = XGB_board.board_object(ptmodel, xmini=dataset.X_data[0], device="cpu", output_dir=logger_path)

New output:/nfs/dust/atlas/user/hteagle/PyAnalysisUtils/ML/NN/notebooks/lightning_logs/version_15/
Got Pytorch model


../modules.py:64: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = torch.nn.functional.softmax(x)## When training/calculating losses using CrossEntropyLoss we should feed it raw logits, it applies softmax itself. When applying/getting probabilites, we should switch this on..annoying


In [15]:
from torch.utils.data import DataLoader
# Load in one big batch using the splits from the model
dataset = bbMeT_NN(root='data', device="cpu", transform=None)
train_loader = DataLoader(dataset, batch_size=len(model.train_indices))
test_loader = DataLoader(dataset, batch_size=len(model.test_indices))
del model

DatasetLoader :: INFO     ::       Attempt to load:/nfs/dust/atlas/user/hteagle/PyAnalysisUtils/ML/NN/data/processed/data_Wh_21_2_112_reco_sig.pt


[]


In [ ]:
board.get_inputs(train_loader, test_loader)# Predict on the train and test datasets for comparison

Pytorch_board :: INFO     ::       Getting input lists


Epoch 1:   0%|          | 0/5 [05:07<?, ?it/s]


1063628it [00:27, 38709.17it/s]
1063628it [00:25, 41050.71it/s]
1063628it [00:26, 39608.01it/s]
1063628it [00:27, 38045.44it/s]
1063624it [00:46, 22917.49it/s]
Pytorch_board :: INFO     ::       Finished getting test inputs


In [ ]:
c = board.draw_train_test_root()
c.Draw()

In [ ]:
%matplotlib inline
auc, curve = board.get_roc(class_ = "Signal")
auc

In [ ]:
curve.ax_.legend("Signal ROC, ACU: %f"%(auc))
curve.plot()